Files conversion
================

Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import pims
from pims import *

In [3]:
import cv2
import os

In [4]:
from pathlib import *
current_dir = Path.cwd()
current_dir

PosixPath('/data/sources/microscope_video_conversion/notebooks')

In [5]:
input_folder = Path("/") / "data" / "IOAN" / "INPUT"
output_folder = Path("/") / "data" / "IOAN" / "OUTPUT"
##input_folder = Path('/media/ioan/Elements/IOAN/PP/exp2/day5/odr-3_ife-2')
##output_folder = Path('/media/ioan/Elements/IOAN/PP_CONVERTED/exp2/day5/odr-3_ife-2')

if(not output_folder.exists()):
    output_folder.mkdir()

In [6]:
def convert(from_file: Path, to_file: Path, fps: int):
    worm = pims.Bioformats(from_file, java_memory='1024m')
    frame = worm[0]
    height, width = frame.shape
    video = cv2.VideoWriter(str(to_file.absolute()), cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height), False)    
    for image in worm:
        image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        video.write(image)
    print(f"{to_file} successfully converted!")
    return to_file

Processing the directory tree with conversion
=============================================

In [7]:
def process(folder: Path, output: Path, skip_if_exists: bool, fps: int = 10):
    print(folder)
    output.mkdir(exist_ok=True)
    for f in folder.iterdir():
        if f.is_dir():
            od: Path = output / f.name
            od.mkdir(exist_ok=True)                        
            process(f, od, skip_if_exists)
        else:
            if "zvi" in f.name:
                to = output / (f.stem+'.mp4')
                if to.exists() and skip_if_exists:
                    print(f"{str(f.absolute())} already exists, skipping...")
                else:                    
                    print(f"converting {f.absolute()} to {to}")
                    #try:
                    convert(f, to, fps)
                    #except:
                    #    print("Exception occured!")                        
                    #    continue
    return folder

In [8]:
process(input_folder, output_folder, True, 10)
print("All conversions finished!")

/data/IOAN/INPUT
converting /data/IOAN/INPUT/w3.zvi to /data/IOAN/OUTPUT/w3.mp4
Exception occured!
converting /data/IOAN/INPUT/w2.zvi to /data/IOAN/OUTPUT/w2.mp4
Exception occured!
All conversions finished!
